In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import f1_score
from typing import Counter
from sklearn.model_selection import GridSearchCV

In [3]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

Combine train and test data for Data Cleaning

In [5]:
main_df = pd.concat([test.assign(ind="test"), train.assign(ind="train")])

In [6]:
main_df.shape

(590540, 435)

In [7]:
main_df.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,ind,isFraud
0,3460689,12245831,325.000,W,12932,361.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test,NaN
1,3415050,10807203,37.257,C,14276,177.0,185.0,mastercard,137.0,credit,...,NaN,NaN,F,F,T,F,mobile,SM-G9600 Build/R16NW,test,NaN
2,3408682,10685267,100.000,R,16506,554.0,150.0,visa,226.0,credit,...,1536x960,match_status:2,T,F,T,F,desktop,Windows,test,NaN
3,3389297,10160809,57.950,W,9500,321.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test,NaN
4,3366334,9492365,34.258,C,15885,545.0,185.0,visa,138.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test,NaN


In [9]:
main_df.isna().sum()

TransactionID          0
TransactionDT          0
TransactionAmt         0
ProductCD              0
card1                  0
                   ...  
id_38             449555
DeviceType        449730
DeviceInfo        471874
ind                    0
isFraud           147635
Length: 435, dtype: int64

# Pre-processing

### Feature engineering of DeviceInfo

Device info contains the brand of device but written in different styles,<br>
So we have just created a different features which integrates the same brands written differently. 

In [48]:
main_df['device_name'] = main_df['DeviceInfo'].str.split('/', expand=True)[0]

In [49]:
main_df.loc[main_df['device_name'].str.contains('SM', na=False), 'device_name'] = 'Samsung'
main_df.loc[main_df['device_name'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung'
main_df.loc[main_df['device_name'].str.contains('GT-', na=False), 'device_name'] = 'Samsung' 
main_df.loc[main_df['device_name'].str.contains('Moto G', na=False), 'device_name'] = 'Motorola'
main_df.loc[main_df['device_name'].str.contains('Moto', na=False), 'device_name'] = 'Motorola'
main_df.loc[main_df['device_name'].str.contains('moto', na=False), 'device_name'] = 'Motorola'
main_df.loc[main_df['device_name'].str.contains('LG-', na=False), 'device_name'] = 'LG'
main_df.loc[main_df['device_name'].str.contains('rv:', na=False), 'device_name'] = 'RV'
main_df.loc[main_df['device_name'].str.contains('HUAWEI', na=False), 'device_name'] = 'Huawei'
main_df.loc[main_df['device_name'].str.contains('ALE-', na=False), 'device_name'] = 'Huawei'
main_df.loc[main_df['device_name'].str.contains('-L', na=False), 'device_name'] = 'Huawei'
main_df.loc[main_df['device_name'].str.contains('Blade', na=False), 'device_name'] = 'ZTE'
main_df.loc[main_df['device_name'].str.contains('BLADE', na=False), 'device_name'] = 'ZTE'
main_df.loc[main_df['device_name'].str.contains('Linux', na=False), 'device_name'] = 'Linux'
main_df.loc[main_df['device_name'].str.contains('XT', na=False), 'device_name'] = 'Sony'
main_df.loc[main_df['device_name'].str.contains('HTC', na=False), 'device_name'] = 'HTC'
main_df.loc[main_df['device_name'].str.contains('ASUS', na=False), 'device_name'] = 'Asus'

In [50]:
main_df.loc[main_df.device_name.isin(main_df.device_name.value_counts()[main_df.device_name.value_counts() < 200].index), 'device_name'] = 'Others'

In [51]:
main_df["device_name"].nunique()

13

### Dropping Columns(Features)

#### TransactionID and TransactionDT

In [52]:
main_df.drop(axis="columns", labels="TransactionID", inplace=True)
main_df.drop(axis="columns", labels="TransactionDT", inplace=True)

#### Categorical

In [53]:
main_df.drop(axis="columns", labels="DeviceInfo", inplace=True)

In [54]:
main_df.drop(axis="columns", labels="DeviceType", inplace=True)

#### Mx

In [56]:
main_df.drop(axis="columns", labels="M3", inplace=True)
main_df.drop(axis="columns", labels="M8", inplace=True)
main_df.drop(axis="columns", labels="M9", inplace=True)

#### Dropping dist1 and dist2

In [58]:
main_df.drop(axis="columns", labels="dist2", inplace=True)

#### Dropping Dxx

In [59]:
main_df.drop(axis="columns", labels="D2", inplace=True)
main_df.drop(axis="columns", labels="D7", inplace=True)

In [60]:
main_df.drop(axis="columns", labels="D12", inplace=True)

# too many null values
main_df.drop(axis="columns", labels="D8", inplace=True)
main_df.drop(axis="columns", labels="D9", inplace=True)
main_df.drop(axis="columns", labels="D13", inplace=True)
main_df.drop(axis="columns", labels="D14", inplace=True)

#### Dropping 'id_xx' columns

In [62]:
main_df.drop(axis="columns", labels="id_38", inplace=True)
main_df.drop(axis="columns", labels="id_34", inplace=True)
main_df.drop(axis="columns", labels="id_33", inplace=True)
main_df.drop(axis="columns", labels="id_31", inplace=True)
main_df.drop(axis="columns", labels="id_27", inplace=True)
main_df.drop(axis="columns", labels="id_26", inplace=True)
main_df.drop(axis="columns", labels="id_25", inplace=True)
main_df.drop(axis="columns", labels="id_24", inplace=True)
main_df.drop(axis="columns", labels="id_23", inplace=True)
main_df.drop(axis="columns", labels="id_22", inplace=True)
main_df.drop(axis="columns", labels="id_21", inplace=True)
main_df.drop(axis="columns", labels="id_12", inplace=True)
main_df.drop(axis="columns", labels="id_08", inplace=True)
main_df.drop(axis="columns", labels="id_07", inplace=True)

#### Dropping 'Vxxx' columns

In [71]:
main_df.drop(axis="columns", labels="V339", inplace=True)
main_df.drop(axis="columns", labels="V338", inplace=True)
main_df.drop(axis="columns", labels="V337", inplace=True)
main_df.drop(axis="columns", labels="V336", inplace=True)
main_df.drop(axis="columns", labels="V335", inplace=True)

In [72]:
main_df.drop(axis="columns", labels="V334", inplace=True)
main_df.drop(axis="columns", labels="V333", inplace=True)
main_df.drop(axis="columns", labels="V332", inplace=True)
main_df.drop(axis="columns", labels="V331", inplace=True)
main_df.drop(axis="columns", labels="V330", inplace=True)

In [73]:
main_df.drop(axis="columns", labels="V329", inplace=True)
main_df.drop(axis="columns", labels="V328", inplace=True)
main_df.drop(axis="columns", labels="V327", inplace=True)
main_df.drop(axis="columns", labels="V326", inplace=True)
main_df.drop(axis="columns", labels="V325", inplace=True)

In [74]:
main_df.drop(axis="columns", labels="V324", inplace=True)
main_df.drop(axis="columns", labels="V323", inplace=True)
main_df.drop(axis="columns", labels="V322", inplace=True)

In [75]:
main_df.drop(axis="columns", labels="V278", inplace=True)
main_df.drop(axis="columns", labels="V277", inplace=True)
main_df.drop(axis="columns", labels="V276", inplace=True)

In [76]:
main_df.drop(axis="columns", labels="V275", inplace=True)
main_df.drop(axis="columns", labels="V274", inplace=True)
main_df.drop(axis="columns", labels="V273", inplace=True)

In [77]:
main_df.drop(axis="columns", labels="V272", inplace=True)
main_df.drop(axis="columns", labels="V271", inplace=True)
main_df.drop(axis="columns", labels="V270", inplace=True)

In [78]:
main_df.drop(axis="columns", labels="V269", inplace=True)
main_df.drop(axis="columns", labels="V268", inplace=True)
main_df.drop(axis="columns", labels="V267", inplace=True)

In [79]:
main_df.drop(axis="columns", labels="V266", inplace=True)
main_df.drop(axis="columns", labels="V265", inplace=True)
main_df.drop(axis="columns", labels="V264", inplace=True)

In [80]:
main_df.drop(axis="columns", labels="V263", inplace=True)
main_df.drop(axis="columns", labels="V262", inplace=True)
main_df.drop(axis="columns", labels="V261", inplace=True)

In [81]:
main_df.drop(axis="columns", labels="V260", inplace=True)
main_df.drop(axis="columns", labels="V259", inplace=True)
main_df.drop(axis="columns", labels="V258", inplace=True)

In [82]:
main_df.drop(axis="columns", labels="V257", inplace=True)
main_df.drop(axis="columns", labels="V256", inplace=True)
main_df.drop(axis="columns", labels="V255", inplace=True)

In [83]:
main_df.drop(axis="columns", labels="V254", inplace=True)
main_df.drop(axis="columns", labels="V253", inplace=True)
main_df.drop(axis="columns", labels="V252", inplace=True)

In [84]:
main_df.drop(axis="columns", labels="V251", inplace=True)
main_df.drop(axis="columns", labels="V250", inplace=True)
main_df.drop(axis="columns", labels="V249", inplace=True)

In [85]:
main_df.drop(axis="columns", labels="V248", inplace=True)
main_df.drop(axis="columns", labels="V247", inplace=True)
main_df.drop(axis="columns", labels="V246", inplace=True)

In [86]:
main_df.drop(axis="columns", labels="V245", inplace=True)
main_df.drop(axis="columns", labels="V244", inplace=True)
main_df.drop(axis="columns", labels="V243", inplace=True)

In [87]:
main_df.drop(axis="columns", labels="V242", inplace=True)
main_df.drop(axis="columns", labels="V241", inplace=True)
main_df.drop(axis="columns", labels="V240", inplace=True)

In [88]:
main_df.drop(axis="columns", labels="V239", inplace=True)
main_df.drop(axis="columns", labels="V238", inplace=True)
main_df.drop(axis="columns", labels="V237", inplace=True)

In [89]:
main_df.drop(axis="columns", labels="V236", inplace=True)
main_df.drop(axis="columns", labels="V235", inplace=True)
main_df.drop(axis="columns", labels="V234", inplace=True)

In [90]:
main_df.drop(axis="columns", labels="V233", inplace=True)
main_df.drop(axis="columns", labels="V232", inplace=True)
main_df.drop(axis="columns", labels="V231", inplace=True)

In [91]:
main_df.drop(axis="columns", labels="V230", inplace=True)
main_df.drop(axis="columns", labels="V229", inplace=True)
main_df.drop(axis="columns", labels="V228", inplace=True)

In [92]:
main_df.drop(axis="columns", labels="V227", inplace=True)
main_df.drop(axis="columns", labels="V226", inplace=True)
main_df.drop(axis="columns", labels="V225", inplace=True)

In [93]:
main_df.drop(axis="columns", labels="V224", inplace=True)
main_df.drop(axis="columns", labels="V223", inplace=True)
main_df.drop(axis="columns", labels="V222", inplace=True)

In [94]:
main_df.drop(axis="columns", labels="V221", inplace=True)
main_df.drop(axis="columns", labels="V220", inplace=True)
main_df.drop(axis="columns", labels="V219", inplace=True)

In [95]:
main_df.drop(axis="columns", labels="V218", inplace=True)
main_df.drop(axis="columns", labels="V217", inplace=True)
main_df.drop(axis="columns", labels="V216", inplace=True)

In [96]:
main_df.drop(axis="columns", labels="V215", inplace=True)
main_df.drop(axis="columns", labels="V214", inplace=True)
main_df.drop(axis="columns", labels="V213", inplace=True)

In [97]:
main_df.drop(axis="columns", labels="V212", inplace=True)
main_df.drop(axis="columns", labels="V211", inplace=True)
main_df.drop(axis="columns", labels="V210", inplace=True)

In [98]:
main_df.drop(axis="columns", labels="V209", inplace=True)
main_df.drop(axis="columns", labels="V208", inplace=True)
main_df.drop(axis="columns", labels="V207", inplace=True)

In [99]:
main_df.drop(axis="columns", labels="V206", inplace=True)
main_df.drop(axis="columns", labels="V205", inplace=True)
main_df.drop(axis="columns", labels="V204", inplace=True)

In [100]:
main_df.drop(axis="columns", labels="V203", inplace=True)
main_df.drop(axis="columns", labels="V202", inplace=True)
main_df.drop(axis="columns", labels="V201", inplace=True)

In [101]:
main_df.drop(axis="columns", labels="V200", inplace=True)
main_df.drop(axis="columns", labels="V199", inplace=True)
main_df.drop(axis="columns", labels="V198", inplace=True)

In [102]:
main_df.drop(axis="columns", labels="V197", inplace=True)
main_df.drop(axis="columns", labels="V196", inplace=True)
main_df.drop(axis="columns", labels="V195", inplace=True)

In [103]:
main_df.drop(axis="columns", labels="V194", inplace=True)
main_df.drop(axis="columns", labels="V193", inplace=True)
main_df.drop(axis="columns", labels="V192", inplace=True)

In [104]:
main_df.drop(axis="columns", labels="V191", inplace=True)
main_df.drop(axis="columns", labels="V190", inplace=True)
main_df.drop(axis="columns", labels="V189", inplace=True)

In [105]:
main_df.drop(axis="columns", labels="V188", inplace=True)
main_df.drop(axis="columns", labels="V187", inplace=True)
main_df.drop(axis="columns", labels="V186", inplace=True)

In [106]:
main_df.drop(axis="columns", labels="V185", inplace=True)
main_df.drop(axis="columns", labels="V184", inplace=True)
main_df.drop(axis="columns", labels="V183", inplace=True)

In [107]:
main_df.drop(axis="columns", labels="V182", inplace=True)
main_df.drop(axis="columns", labels="V181", inplace=True)
main_df.drop(axis="columns", labels="V180", inplace=True)

In [108]:
main_df.drop(axis="columns", labels="V179", inplace=True)
main_df.drop(axis="columns", labels="V178", inplace=True)
main_df.drop(axis="columns", labels="V177", inplace=True)

In [109]:
main_df.drop(axis="columns", labels="V176", inplace=True)
main_df.drop(axis="columns", labels="V175", inplace=True)
main_df.drop(axis="columns", labels="V174", inplace=True)

In [110]:
main_df.drop(axis="columns", labels="V173", inplace=True)
main_df.drop(axis="columns", labels="V172", inplace=True)
main_df.drop(axis="columns", labels="V171", inplace=True)

In [111]:
main_df.drop(axis="columns", labels="V170", inplace=True)
main_df.drop(axis="columns", labels="V169", inplace=True)
main_df.drop(axis="columns", labels="V168", inplace=True)

In [112]:
main_df.drop(axis="columns", labels="V167", inplace=True)
main_df.drop(axis="columns", labels="V166", inplace=True)
main_df.drop(axis="columns", labels="V165", inplace=True)

In [113]:
main_df.drop(axis="columns", labels="V164", inplace=True)
main_df.drop(axis="columns", labels="V163", inplace=True)
main_df.drop(axis="columns", labels="V162", inplace=True)

In [114]:
main_df.drop(axis="columns", labels="V161", inplace=True)
main_df.drop(axis="columns", labels="V160", inplace=True)
main_df.drop(axis="columns", labels="V159", inplace=True)

In [115]:
main_df.drop(axis="columns", labels="V158", inplace=True)
main_df.drop(axis="columns", labels="V157", inplace=True)
main_df.drop(axis="columns", labels="V156", inplace=True)

In [116]:
main_df.drop(axis="columns", labels="V155", inplace=True)
main_df.drop(axis="columns", labels="V154", inplace=True)
main_df.drop(axis="columns", labels="V153", inplace=True)

In [117]:
main_df.drop(axis="columns", labels="V152", inplace=True)
main_df.drop(axis="columns", labels="V151", inplace=True)
main_df.drop(axis="columns", labels="V150", inplace=True)

In [118]:
main_df.drop(axis="columns", labels="V149", inplace=True)
main_df.drop(axis="columns", labels="V148", inplace=True)
main_df.drop(axis="columns", labels="V147", inplace=True)

In [119]:
main_df.drop(axis="columns", labels="V146", inplace=True)
main_df.drop(axis="columns", labels="V145", inplace=True)
main_df.drop(axis="columns", labels="V144", inplace=True)

In [120]:
main_df.drop(axis="columns", labels="V143", inplace=True)
main_df.drop(axis="columns", labels="V142", inplace=True)
main_df.drop(axis="columns", labels="V141", inplace=True)

In [121]:
main_df.drop(axis="columns", labels="V140", inplace=True)
main_df.drop(axis="columns", labels="V139", inplace=True)
main_df.drop(axis="columns", labels="V138", inplace=True)

In [122]:
main_df.drop(axis="columns", labels="V52", inplace=True)
main_df.drop(axis="columns", labels="V51", inplace=True)
main_df.drop(axis="columns", labels="V50", inplace=True)

In [123]:
main_df.drop(axis="columns", labels="V49", inplace=True)
main_df.drop(axis="columns", labels="V48", inplace=True)
main_df.drop(axis="columns", labels="V47", inplace=True)

In [124]:
main_df.drop(axis="columns", labels="V46", inplace=True)
main_df.drop(axis="columns", labels="V45", inplace=True)
main_df.drop(axis="columns", labels="V44", inplace=True)

In [125]:
main_df.drop(axis="columns", labels="V43", inplace=True)
main_df.drop(axis="columns", labels="V42", inplace=True)
main_df.drop(axis="columns", labels="V41", inplace=True)

In [126]:
main_df.drop(axis="columns", labels="V40", inplace=True)
main_df.drop(axis="columns", labels="V39", inplace=True)
main_df.drop(axis="columns", labels="V38", inplace=True)

In [127]:
main_df.drop(axis="columns", labels="V37", inplace=True)
main_df.drop(axis="columns", labels="V36", inplace=True)
main_df.drop(axis="columns", labels="V35", inplace=True)

In [128]:
main_df.drop(axis="columns", labels="V11", inplace=True)
main_df.drop(axis="columns", labels="V10", inplace=True)
main_df.drop(axis="columns", labels="V9", inplace=True)

In [129]:
main_df.drop(axis="columns", labels="V8", inplace=True)
main_df.drop(axis="columns", labels="V7", inplace=True)
main_df.drop(axis="columns", labels="V6", inplace=True)

In [130]:
main_df.drop(axis="columns", labels="V5", inplace=True)
main_df.drop(axis="columns", labels="V4", inplace=True)
main_df.drop(axis="columns", labels="V3", inplace=True)

In [131]:
main_df.drop(axis="columns", labels="V2", inplace=True)
main_df.drop(axis="columns", labels="V1", inplace=True)

### Replacing NULL values by something

In [132]:
main_df["isFraud"].isna().sum()
main_df["isFraud"].fillna(0, inplace=True)

#### For Vxxx columns

If more outliers than replace by median

If less outliers replace by mean

In [133]:
main_df["V321"].fillna(main_df["V321"].median(), inplace=True)
main_df["V320"].fillna(main_df["V320"].median(), inplace=True)
main_df["V319"].fillna(main_df["V319"].median(), inplace=True)
main_df["V318"].fillna(main_df["V318"].median(), inplace=True)
main_df["V317"].fillna(main_df["V317"].median(), inplace=True)

In [135]:
main_df["V316"].fillna(main_df["V316"].median(), inplace=True)
main_df["V315"].fillna(main_df["V315"].median(), inplace=True)
main_df["V314"].fillna(main_df["V314"].median(), inplace=True)
main_df["V313"].fillna(main_df["V313"].median(), inplace=True)
main_df["V312"].fillna(main_df["V312"].median(), inplace=True)

In [136]:
main_df["V311"].fillna(main_df["V311"].median(), inplace=True)
main_df["V310"].fillna(main_df["V310"].median(), inplace=True)
main_df["V309"].fillna(main_df["V309"].median(), inplace=True)
main_df["V308"].fillna(main_df["V308"].median(), inplace=True)
main_df["V307"].fillna(main_df["V307"].median(), inplace=True)

In [137]:
main_df["V306"].fillna(main_df["V306"].median(), inplace=True)
main_df["V305"].fillna(main_df["V305"].median(), inplace=True)
main_df["V304"].fillna(main_df["V304"].median(), inplace=True)
main_df["V303"].fillna(main_df["V303"].median(), inplace=True)
main_df["V302"].fillna(main_df["V302"].median(), inplace=True)

In [138]:
main_df["V301"].fillna(main_df["V301"].median(), inplace=True)
main_df["V300"].fillna(main_df["V300"].median(), inplace=True)
main_df["V299"].fillna(main_df["V299"].median(), inplace=True)
main_df["V298"].fillna(main_df["V298"].median(), inplace=True)
main_df["V297"].fillna(main_df["V297"].median(), inplace=True)

In [139]:
main_df["V296"].fillna(main_df["V296"].median(), inplace=True)
main_df["V295"].fillna(main_df["V295"].median(), inplace=True)
main_df["V294"].fillna(main_df["V294"].median(), inplace=True)
main_df["V293"].fillna(main_df["V293"].median(), inplace=True)
main_df["V292"].fillna(main_df["V292"].median(), inplace=True)

In [141]:
main_df["V291"].fillna(main_df["V291"].median(), inplace=True)
main_df["V290"].fillna(main_df["V290"].median(), inplace=True)
main_df["V289"].fillna(main_df["V289"].median(), inplace=True)
main_df["V288"].fillna(main_df["V288"].median(), inplace=True)
main_df["V287"].fillna(main_df["V287"].median(), inplace=True)

In [143]:
main_df["V286"].fillna(main_df["V286"].median(), inplace=True)
main_df["V285"].fillna(main_df["V285"].median(), inplace=True)
main_df["V284"].fillna(main_df["V284"].median(), inplace=True)

In [144]:
main_df["V281"].fillna(main_df["V281"].median(), inplace=True)
main_df["V280"].fillna(main_df["V280"].median(), inplace=True)
main_df["V279"].fillna(main_df["V279"].median(), inplace=True)

In [146]:
main_df["V137"].fillna(main_df["V137"].median(), inplace=True)
main_df["V136"].fillna(main_df["V136"].median(), inplace=True)
main_df["V135"].fillna(main_df["V135"].median(), inplace=True)

In [147]:
main_df["V134"].fillna(main_df["V134"].median(), inplace=True)
main_df["V133"].fillna(main_df["V133"].median(), inplace=True)
main_df["V132"].fillna(main_df["V132"].median(), inplace=True)

In [148]:
main_df["V131"].fillna(main_df["V131"].median(), inplace=True)
main_df["V130"].fillna(main_df["V130"].median(), inplace=True)
main_df["V129"].fillna(main_df["V129"].median(), inplace=True)

In [149]:
main_df["V128"].fillna(main_df["V128"].median(), inplace=True)
main_df["V127"].fillna(main_df["V127"].median(), inplace=True)
main_df["V126"].fillna(main_df["V126"].median(), inplace=True)

In [150]:
main_df["V125"].fillna(main_df["V125"].median(), inplace=True)
main_df["V124"].fillna(main_df["V124"].median(), inplace=True)
main_df["V123"].fillna(main_df["V123"].median(), inplace=True)

In [151]:
main_df["V122"].fillna(main_df["V122"].median(), inplace=True)
main_df["V121"].fillna(main_df["V121"].median(), inplace=True)
main_df["V120"].fillna(main_df["V120"].median(), inplace=True)

In [153]:
main_df["V119"].fillna(main_df["V119"].median(), inplace=True)
main_df["V118"].fillna(main_df["V118"].median(), inplace=True)
main_df["V117"].fillna(main_df["V117"].median(), inplace=True)

In [154]:
main_df["V116"].fillna(main_df["V116"].median(), inplace=True)
main_df["V115"].fillna(main_df["V115"].median(), inplace=True)
main_df["V114"].fillna(main_df["V114"].median(), inplace=True)

In [155]:
main_df["V113"].fillna(main_df["V113"].median(), inplace=True)
main_df["V112"].fillna(main_df["V112"].median(), inplace=True)
main_df["V111"].fillna(main_df["V111"].median(), inplace=True)

In [156]:
main_df["V110"].fillna(main_df["V110"].median(), inplace=True)
main_df["V109"].fillna(main_df["V109"].median(), inplace=True)
main_df["V108"].fillna(main_df["V108"].median(), inplace=True)

In [157]:
main_df["V107"].fillna(main_df["V107"].median(), inplace=True)
main_df["V106"].fillna(main_df["V106"].median(), inplace=True)
main_df["V105"].fillna(main_df["V105"].median(), inplace=True)

In [158]:
main_df["V104"].fillna(main_df["V104"].median(), inplace=True)
main_df["V103"].fillna(main_df["V103"].median(), inplace=True)
main_df["V102"].fillna(main_df["V102"].median(), inplace=True)

In [159]:
main_df["V101"].fillna(main_df["V101"].median(), inplace=True)
main_df["V100"].fillna(main_df["V100"].median(), inplace=True)
main_df["V99"].fillna(main_df["V99"].median(), inplace=True)

In [161]:
main_df["V98"].fillna(main_df["V98"].median(), inplace=True)
main_df["V97"].fillna(main_df["V97"].median(), inplace=True)
main_df["V96"].fillna(main_df["V96"].median(), inplace=True)

In [162]:
main_df["V95"].fillna(main_df["V95"].median(), inplace=True)
main_df["V282"].fillna(main_df["V282"].median(), inplace=True)
main_df["V283"].fillna(main_df["V283"].median(), inplace=True)

#### Every column except Vxxx

In [163]:
main_df["dist1"].fillna(main_df["dist1"].median(), inplace=True)

In [164]:
main_df["card4"].fillna("visa", inplace=True)
main_df["card6"].fillna("debit", inplace=True)
main_df["card2"].fillna(main_df["card2"].median(), inplace=True)
main_df["card3"].fillna(main_df["card3"].median(), inplace=True)
main_df["card5"].fillna(main_df["card5"].median(), inplace=True)

In [165]:
main_df["addr1"].fillna(main_df["addr1"].median(), inplace=True)
main_df["addr2"].fillna(main_df["addr2"].median(), inplace=True)

In [166]:
main_df["D1"].fillna(main_df["D1"].median(), inplace=True)
main_df["D3"].fillna(main_df["D3"].median(), inplace=True)
main_df["D4"].fillna(main_df["D4"].median(), inplace=True)
main_df["D5"].fillna(main_df["D5"].median(), inplace=True)
main_df["D6"].fillna(main_df["D6"].median(), inplace=True)
main_df["D10"].fillna(main_df["D10"].median(), inplace=True)
main_df["D11"].fillna(main_df["D11"].median(), inplace=True)
main_df["D15"].fillna(main_df["D15"].median(), inplace=True)

gmail.com is the mode of P_emaildomain and R_emaildomain

In [168]:
main_df["R_emaildomain"].fillna("gmail.com", inplace=True)
main_df["P_emaildomain"].fillna("gmail.com", inplace=True)

In [169]:
main_df["M1"].fillna("T", inplace=True)
main_df["M2"].fillna("T", inplace=True)
main_df["M4"].fillna("M0", inplace=True)
main_df["M5"].fillna("F", inplace=True)
main_df["M6"].fillna("F", inplace=True)
main_df["M7"].fillna("F", inplace=True)

In [170]:
main_df["id_01"].fillna(main_df["id_01"].median(), inplace=True)
main_df["id_02"].fillna(main_df["id_02"].median(), inplace=True)
main_df["id_03"].fillna(main_df["id_03"].median(), inplace=True)
main_df["id_04"].fillna(main_df["id_04"].median(), inplace=True)
main_df["id_05"].fillna(main_df["id_05"].median(), inplace=True)
main_df["id_06"].fillna(main_df["id_06"].median(), inplace=True)
main_df["id_09"].fillna(main_df["id_09"].median(), inplace=True)
main_df["id_10"].fillna(main_df["id_10"].median(), inplace=True)
main_df["id_11"].fillna(main_df["id_11"].median(), inplace=True)
main_df["id_13"].fillna(main_df["id_13"].median(), inplace=True)
main_df["id_14"].fillna(main_df["id_14"].median(), inplace=True)
main_df["id_17"].fillna(main_df["id_17"].median(), inplace=True)
main_df["id_18"].fillna(main_df["id_18"].median(), inplace=True)
main_df["id_19"].fillna(main_df["id_19"].median(), inplace=True)
main_df["id_20"].fillna(main_df["id_20"].median(), inplace=True)
main_df["id_32"].fillna(main_df["id_32"].median(), inplace=True)

In [171]:
main_df["id_15"].fillna("Unknown", inplace=True)
main_df["id_16"].fillna("Unknown", inplace=True)
main_df["id_28"].fillna("Unknown", inplace=True)
main_df["id_29"].fillna("Unknown", inplace=True)
main_df["id_30"].fillna("Windows 10", inplace=True)
main_df["id_35"].fillna("None", inplace=True)
main_df["id_36"].fillna("F", inplace=True)
main_df["id_37"].fillna("T", inplace=True)

#### For Vxxx feature

If more outliers than replace by median

If less outliers replace by mean

In [173]:
main_df["V94"].fillna(main_df["V94"].median(), inplace=True)

In [174]:
main_df["V93"].fillna(main_df["V93"].median(), inplace=True)
main_df["V94"].fillna(main_df["V94"].median(), inplace=True)
main_df["V92"].fillna(main_df["V92"].median(), inplace=True)
main_df["V91"].fillna(main_df["V91"].median(), inplace=True)
main_df["V90"].fillna(main_df["V90"].median(), inplace=True)

In [175]:
main_df["V89"].fillna(main_df["V89"].median(), inplace=True)
main_df["V88"].fillna(main_df["V88"].median(), inplace=True)
main_df["V87"].fillna(main_df["V87"].median(), inplace=True)
main_df["V86"].fillna(main_df["V86"].median(), inplace=True)
main_df["V85"].fillna(main_df["V85"].median(), inplace=True)

In [176]:
main_df["V84"].fillna(main_df["V84"].median(), inplace=True)
main_df["V83"].fillna(main_df["V83"].median(), inplace=True)
main_df["V82"].fillna(main_df["V82"].median(), inplace=True)
main_df["V81"].fillna(main_df["V81"].median(), inplace=True)
main_df["V80"].fillna(main_df["V80"].median(), inplace=True)

In [ ]:
main_df["V79"].fillna(main_df["V79"].median(), inplace=True)
main_df["V78"].fillna(main_df["V78"].median(), inplace=True)
main_df["V77"].fillna(main_df["V77"].median(), inplace=True)
main_df["V76"].fillna(main_df["V76"].median(), inplace=True)
main_df["V75"].fillna(main_df["V75"].median(), inplace=True)

In [ ]:
main_df["V74"].fillna(main_df["V74"].median(), inplace=True)
main_df["V73"].fillna(main_df["V73"].median(), inplace=True)
main_df["V72"].fillna(main_df["V72"].median(), inplace=True)
main_df["V71"].fillna(main_df["V71"].median(), inplace=True)
main_df["V70"].fillna(main_df["V70"].median(), inplace=True)

In [ ]:
main_df["V69"].fillna(main_df["V69"].median(), inplace=True)
main_df["V68"].fillna(main_df["V68"].median(), inplace=True)
main_df["V67"].fillna(main_df["V67"].median(), inplace=True)
main_df["V66"].fillna(main_df["V66"].median(), inplace=True)
main_df["V65"].fillna(main_df["V65"].median(), inplace=True)

In [ ]:
main_df["V64"].fillna(main_df["V64"].median(), inplace=True)
main_df["V63"].fillna(main_df["V63"].median(), inplace=True)
main_df["V62"].fillna(main_df["V62"].median(), inplace=True)
main_df["V61"].fillna(main_df["V61"].median(), inplace=True)
main_df["V60"].fillna(main_df["V60"].median(), inplace=True)

In [ ]:
main_df["V59"].fillna(main_df["V59"].median(), inplace=True)
main_df["V58"].fillna(main_df["V58"].median(), inplace=True)
main_df["V57"].fillna(main_df["V57"].median(), inplace=True)
main_df["V56"].fillna(main_df["V56"].median(), inplace=True)
main_df["V55"].fillna(main_df["V55"].median(), inplace=True)

In [ ]:
main_df["V54"].fillna(main_df["V54"].median(), inplace=True)
main_df["V53"].fillna(main_df["V53"].median(), inplace=True)

In [ ]:
main_df["V34"].fillna(main_df["V34"].median(), inplace=True)
main_df["V33"].fillna(main_df["V33"].median(), inplace=True)
main_df["V32"].fillna(main_df["V32"].median(), inplace=True)
main_df["V31"].fillna(main_df["V31"].median(), inplace=True)
main_df["V30"].fillna(main_df["V30"].median(), inplace=True)

In [ ]:
main_df["V29"].fillna(main_df["V29"].median(), inplace=True)
main_df["V28"].fillna(main_df["V28"].median(), inplace=True)
main_df["V27"].fillna(main_df["V27"].median(), inplace=True)
main_df["V26"].fillna(main_df["V26"].median(), inplace=True)
main_df["V25"].fillna(main_df["V25"].median(), inplace=True)

In [ ]:
main_df["V24"].fillna(main_df["V24"].median(), inplace=True)
main_df["V23"].fillna(main_df["V23"].median(), inplace=True)
main_df["V22"].fillna(main_df["V22"].median(), inplace=True)
main_df["V21"].fillna(main_df["V21"].median(), inplace=True)
main_df["V20"].fillna(main_df["V20"].median(), inplace=True)

In [ ]:
main_df["V19"].fillna(main_df["V19"].median(), inplace=True)
main_df["V18"].fillna(main_df["V18"].median(), inplace=True)
main_df["V17"].fillna(main_df["V17"].median(), inplace=True)
main_df["V16"].fillna(main_df["V16"].median(), inplace=True)
main_df["V15"].fillna(main_df["V15"].median(), inplace=True)
main_df["V14"].fillna(main_df["V14"].median(), inplace=True)
main_df["V13"].fillna(main_df["V13"].median(), inplace=True)
main_df["V12"].fillna(main_df["V12"].median(), inplace=True)

### Label encoding

In [190]:
le = LabelEncoder()

Label encoding the features with high unique values so that the number of features does not increate drastically

In [192]:
main_df['id_30'] = le.fit_transform(main_df['id_30'])
main_df['R_emaildomain'] = le.fit_transform(main_df['R_emaildomain'])
main_df['P_emaildomain'] = le.fit_transform(main_df['P_emaildomain'])

### One-Hot encoding

In [228]:
column_names_to_one_hot = ["ProductCD", "card4", "card6", "M1", "M2", "M4", "M5", "M6", "M7", "id_15", "id_16", "id_28", "id_29", "id_35", "id_36", "id_37", "device_name"]

In [229]:
main_df = pd.get_dummies(main_df, columns=column_names_to_one_hot)

In [230]:
main_df.head()

,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,C1,C2,...,id_29_Found,id_29_NotFound,id_29_Unknown,id_35_F,id_35_None,id_35_T,id_36_F,id_36_T,id_37_F,id_37_T
0,325.000,12932,361.0,150.0,226.0,126.0,87.0,24.0,3.0,1.0,...,0,0,1,0,1,0,1,0,0,1
1,37.257,14276,177.0,185.0,137.0,299.0,87.0,8.0,0.0,1.0,...,0,1,0,1,0,0,1,0,0,1
2,100.000,16506,554.0,150.0,226.0,299.0,87.0,8.0,1.0,1.0,...,0,1,0,0,0,1,1,0,0,1
3,57.950,9500,321.0,150.0,226.0,441.0,87.0,295.0,1.0,1.0,...,0,0,1,0,1,0,1,0,0,1
4,34.258,15885,545.0,185.0,138.0,284.0,60.0,8.0,1.0,1.0,...,0,0,1,0,1,0,1,0,0,1


### Seperating test-train data

In [231]:
test_standard, train_standard = main_df[main_df["ind"].eq("test")], main_df[main_df["ind"].eq("train")]

In [232]:
test_standard.drop(axis="columns", labels="ind", inplace=True)
train_standard.drop(axis="columns", labels="ind", inplace=True)

C:\Users\Prem Shah\AppData\Local\Temp\ipykernel_16288\3193864518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_standard.drop(axis="columns", labels="ind", inplace=True)
C:\Users\Prem Shah\AppData\Local\Temp\ipykernel_16288\3193864518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_standard.drop(axis="columns", labels="ind", inplace=True)


Storing pre-processed data as csv files

In [ ]:
train_standard.to_csv('pre-processed_train.csv', index=False)
test_standard.to_csv('pre_precessed_test.csv', index=False)

### Splitting into feature and target columns

In [233]:
X_train = train_standard.drop(axis="columns", labels="isFraud").to_numpy().astype(np.float64)
y_train = train_standard["isFraud"].to_numpy().astype(np.float64)
X_test = test_standard.drop(axis="columns", labels="isFraud").to_numpy().astype(np.float64)

# Re-Sampling

In [234]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

Our dataset is highly imbalanced, so we need to make a balance between number of fraud transaction data and non-fraud transaction data

In [235]:
over = RandomOverSampler(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)

In [236]:
pipeline = Pipeline(steps=[('o', over), ('u', under)])
train_X_re, train_y_re = pipeline.fit_resample(X_train, y_train)

In [220]:
train_y_re.shape, train_X_re.shape

((128220,), (128220, 258))

In [237]:
print(Counter(y_train).keys())
print(Counter(y_train).values())

dict_keys([0.0, 1.0])
dict_values([427408, 15497])


Ratio: 0.03625 of fraud to non fraud

In [222]:
print(Counter(train_y_re).keys())
print(Counter(train_y_re).values())

dict_keys([0.0, 1.0])
dict_values([85480, 42740])


Ratio: 0.5 of fraud to non fraud